# Analysis and Visualization

Let's create a visualization of a single meme:

## 0. Setup

In [1]:
import os
import os.path

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [2]:
# Parameters

# Folders on local machine where to create the output and temporary files:
input_path = "wikidata"
output_path = "projects"
project_name = "tutorial-kypher"

In [3]:
big_files=["label"]

additional_files = {
    "P31": "derived.P31.tsv.gz",
    "items": "claims.wikibase-item.tsv.gz",
    "P1963": "derived.P1963computed.count.star.tsv.gz",
    "external": "claims.external-id.tsv.gz",
    "indegree": "metadata.in_degree.tsv.gz",
    "outdegree": "metadata.out_degree.tsv.gz",
    "pagerank": "metadata.pagerank.directed.tsv.gz"
}

ck = ConfigureKGTK(big_files)
ck.configure_kgtk(input_graph_path=input_path, 
                  output_path=output_path, 
                  project_name=project_name,
                  additional_files=additional_files)

User home: /Users/filipilievski
Current dir: /Users/filipilievski/mcs/imkg
KGTK dir: /Users/filipilievski/mcs
Use-cases dir: /Users/filipilievski/mcs/use-cases


In [4]:
ck.print_env_variables()

USE_CASES_DIR: /Users/filipilievski/mcs/use-cases
GRAPH: wikidata
KGTK_GRAPH_CACHE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
STORE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
TEMP: projects/tutorial-kypher/temp.tutorial-kypher
kgtk: kgtk
KGTK_LABEL_FILE: wikidata/labels.en.tsv.gz
OUT: projects/tutorial-kypher
EXAMPLES_DIR: /Users/filipilievski/mcs/examples
kypher: kgtk query --graph-cache projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
label: wikidata/labels.en.tsv.gz
P31: wikidata/derived.P31.tsv.gz
items: wikidata/claims.wikibase-item.tsv.gz
P1963: wikidata/derived.P1963computed.count.star.tsv.gz
external: wikidata/claims.external-id.tsv.gz
indegree: wikidata/metadata.in_degree.tsv.gz
outdegree: wikidata/metadata.out_degree.tsv.gz
pagerank: wikidata/metadata.pagerank.directed.tsv.gz


In [38]:
!gzcat "$TEMP/imkg.kgtk.gz" | grep "mordor"

imgflipi:2494xe	imgflipr:alt_text	hope the water in mordor don\'t brian him. \|  WINS A TICKET TO A FOREIGN COUNTRY; MORDOR \| image tagged in memes,bad luck brian \| made w/ Imgflip meme maker
imgflipi:2494xe	imgflipr:title	hope the water in mordor don\'t brian him.
imgflipi:627jk1	imgflipr:alt_text	Man going to mordor \|  THAT OTHER GUY; BOROMIR; MORDOR \| image tagged in memes,distracted boyfriend \| made w/ Imgflip meme maker
imgflipi:627jk1	imgflipr:title	Man going to mordor


# 1. Find memes that show the connection between KYM, ImgFlip and Wikidata

In [36]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
            --match '()-[:`m4s:sameAs`]->(k), \
                    (k)-[]->(kinfo)'

node1	label	node2	node1	label	node2


In [14]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
            --match '(:`imgflipmeme:135990973/distracted-boyfriend-anime-re-zero`)-[:`kym:fromAbout`]->(info1), \
                    (:`kym:distracted-boyfriend`)-[`kym:fromImage`]->(info2), \
                    (info1)-[r]->(info2)' \
            --where 'info2!="0.5"' \
            --return 'distinct info1, r.label, info2' \
            --limit 500

node2	label	node2


In [7]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
            --match '(:`kym:one-does-not-simply-walk-into-mordor`)-[`m4s:fromAbout`]->(info1), \
                    (:`kym:one-does-not-simply-walk-into-mordor`)-[r2]->(info2), \
                    (info1)-[r]->(info2)' \
            --where 'info2!="0.5"' \
            --return 'distinct info1, r.label, info2, r2.label' \
            --limit 5

node2	label	node2	label


## 1. Visualize most relations for the Distracted-Boyfriend meme

In [5]:
!kgtk query -i $TEMP/templates.kgtk.gz \
            --match '(:`kym:distracted-boyfriend`)-[r]->()' \
             --where 'r.label in ["kym:parent", "kym:child", "kym:year", "rdf:type", "m4s:fromAbout", "m4s:fromTags", "m4s:fromImage"]' \
            -o $TEMP/db_subject.kgtk.gz

Create node file:

In [6]:
!kgtk query -i $TEMP/labelfile.kgtk.gz -i $TEMP/db_subject.kgtk.gz \
            --match 'db: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as id, l as label' / deduplicate \
            -o $TEMP/nodefile.kgtk.gz 

In [9]:
kgtk("""visualize-graph 
        -i $TEMP/db_subject.kgtk.gz
        --node-file $TEMP/nodefile.kgtk.gz
        --show-text above
        --tooltip-column label
        --direction arrow
        --edge-color-column label
        --edge-color-style d3.schemeDark2
        -o distracted.graph.html""")

## 2. Visualize children and parents for the TLDR meme

In [16]:
!kgtk query -i $TEMP/templates.kgtk.gz \
            --match '(n)-[r]->(n2)' \
            --where 'r.label in ["kym:parent", "kym:child"] and (n="kym:tldr" or n2="kym:tldr")' \
            -o $TEMP/db_subject.kgtk.gz

In [19]:
kgtk("""visualize-graph 
        -i $TEMP/db_subject.kgtk.gz
        --show-text above
        --tooltip-column label
        --direction arrow
        --edge-color-column label
        --edge-color-style d3.schemeDark2
        -o tldr.graph.html""")

## 3. Visualize all sibling relations

In [7]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
            --match '()-[r:`kym:sibling`]->()' \
            -o $TEMP/siblings.kgtk.gz

In [8]:
kgtk("""visualize-graph 
        -i $TEMP/siblings.kgtk.gz
        --direction arrow
        -o sibling.graph.html""")

## 4. Analyze graph

### IMKG

In [27]:
!kgtk cat -i $TEMP/templates.kgtk.gz -i $TEMP/wikidata_memes.kgtk.gz -i $TEMP/wikidata_ent.kgtk.gz -o $TEMP/templates_with_wd.kgtk.gz

In [5]:
!kgtk graph-statistics \
     -i $TEMP/imkg.kgtk.gz \
     --log-file $TEMP/meme_summary.txt \
     --output-statistics-only \
     -o $TEMP/meme_stats.tsv


(kgtk:5678): Gtk-WARNING **: 11:32:34.211: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[5678]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x19f89a960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1b3d426b0). One of the two will be used. Which one is undefined.


In [6]:
!cat $TEMP/meme_summary.txt

graph loaded! It has 646690 nodes and 2230499 edges

*** Top relations:
m4s:fromCaption	460177
imgflipr:alt_text	160461
imgflipr:image_url	160461
imgflipr:template	160461
imgflipr:templateId	160461
imgflipr:template_title	160461
imgflipr:title	160461
imgflipr:upvote_count	160461
imgflipr:view_count	160461
imgflipr:author	137923

*** Degrees:
in degree stats: mean=3.449101, std=0.301658, max=1
out degree stats: mean=3.449101, std=0.011192, max=1
total degree stats: mean=6.898202, std=0.301999, max=1

*** PageRank
Max pageranks
15	Q2927074	0.009397
101	Q30	0.006346
621526	Q336	0.005086
630289	Q9081	0.004848
1560	Bad Luck Brian	0.004195

*** HITS
HITS hubs
15	Q2927074	0.546047
1560	Bad Luck Brian	0.524021
503	Q18738659	0.290511
1558	imgflipmeme:Bad-Luck-Brian	0.290051
1559	61585	0.290043
HITS auth
545738	imgflipi:izyph	0.005110
46211	imgflipi:183a7b	0.005105
555539	imgflipi:l4j55	0.005103
303752	imgflipi:33jv3i	0.005095
600279	imgflipi:vumoh	0.005093


In [9]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
    --match '(n)-[r]->()' \
    --return 'count (distinct r.label)'

count(DISTINCT graph_27_c1."label")
650


### For KYM

In [10]:
!kgtk graph-statistics \
     -i $TEMP/templates.kgtk.gz \
     --log-file $TEMP/kym_summary.txt \
     --output-statistics-only \
     -o $TEMP/kym_stats.tsv


(kgtk:5758): Gtk-WARNING **: 11:36:58.594: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[5758]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x19858b960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1aca4c6b0). One of the two will be used. Which one is undefined.


In [11]:
!cat $TEMP/kym_summary.txt

graph loaded! It has 31645 nodes and 211889 edges

*** Top relations:
kym:sibling	111053
m4s:fromImage	53794
kym:tag	11050
m4s:fromAbout	5403
m4s:fromTags	3733
kym:spread	3070
m4s:structured_uri	2812
m4s:structured_value	2812
kym:origin	2479
rdf:type	2234

*** Degrees:
in degree stats: mean=6.695813, std=0.191553, max=1
out degree stats: mean=6.695813, std=0.185121, max=1
total degree stats: mean=13.391626, std=0.318774, max=1

*** PageRank
Max pageranks
28886	http://www.w3.org/2001/XMLSchema#timestamp	0.033271
570	kym:Meme	0.000454
487	0.5	0.000430
556	Q2927074	0.000419
561	Q4868296	0.000389

*** HITS
HITS hubs
570	kym:Meme	0.087234
487	0.5	0.083446
556	Q2927074	0.073038
561	Q4868296	0.070239
1159	kym:lolspeak-chanspeak	0.064137
HITS auth
1112	kym:fap	0.076776
1094	kym:derp	0.076504
1083	kym:cool-story-bro	0.075998
1252	kym:verbose-classy-memes	0.075936
1178	kym:noice	0.075900


In [12]:
!kgtk query -i $TEMP/templates.kgtk.gz \
    --match '(n)-[r]->()' \
    --return 'count (distinct r.label)'

count(DISTINCT graph_20_c1."label")
22


### For ImgFlip

In [13]:
!kgtk graph-statistics \
     -i $TEMP/instances.kgtk.gz \
     --log-file $TEMP/imgflip_summary.txt \
     --output-statistics-only \
     -o $TEMP/imgflip_stats.tsv


(kgtk:5787): Gtk-WARNING **: 11:38:16.549: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[5787]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x11130c960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1ad9ee6b0). One of the two will be used. Which one is undefined.


In [14]:
!cat $TEMP/imgflip_summary.txt

graph loaded! It has 616850 nodes and 1881794 edges

*** Top relations:
m4s:fromCaption	460177
imgflipr:alt_text	160461
imgflipr:image_url	160461
imgflipr:template	160461
imgflipr:templateId	160461
imgflipr:template_title	160461
imgflipr:title	160461
imgflipr:upvote_count	160461
imgflipr:view_count	160461
imgflipr:author	137923

*** Degrees:
in degree stats: mean=3.050651, std=0.313352, max=1
out degree stats: mean=3.050651, std=0.006613, max=1
total degree stats: mean=6.101302, std=0.313373, max=1

*** PageRank
Max pageranks
14	Q2927074	0.011729
160	Bad Luck Brian	0.005716
141262	Distracted Boyfriend	0.004670
35	Disaster Girl	0.003432
55	First World Problems	0.003304

*** HITS
HITS hubs
14	Q2927074	0.541981
160	Bad Luck Brian	0.526372
162	Q18738659	0.291693
158	imgflipmeme:Bad-Luck-Brian	0.291330
159	61585	0.291330
HITS auth
544546	imgflipi:izyph	0.005127
44879	imgflipi:183a7b	0.005120
554347	imgflipi:l4j55	0.005119
302519	imgflipi:33jv3i	0.005111
599089	imgflipi:vumoh	0.005110


In [15]:
!kgtk query -i $TEMP/instances.kgtk.gz \
    --match '(n)-[r]->()' \
    --return 'count (distinct r.label)'

count(DISTINCT graph_31_c1."label")
11


### Wikidata

In [16]:
!kgtk cat -i $TEMP/wikidata_memes.kgtk.gz -i $TEMP/wikidata_ent.kgtk.gz -o $TEMP/wd.kgtk.gz

In [17]:
!kgtk graph-statistics \
     -i $TEMP/wd.kgtk.gz \
     --log-file $TEMP/wd_summary.txt \
     --output-statistics-only \
     -o $TEMP/wd_stats.tsv


(kgtk:5862): Gtk-WARNING **: 11:41:02.783: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[5862]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x199d76960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1ae2a56b0). One of the two will be used. Which one is undefined.


In [18]:
!cat $TEMP/wd_summary.txt

graph loaded! It has 27024 nodes and 136577 edges

*** Top relations:
P31	18090
P136	8558
P495	6942
P106	6030
P17	5572
P407	4363
P279	4312
P530	3713
P21	3657
P27	2975

*** Degrees:
in degree stats: mean=5.053915, std=0.505252, max=1
out degree stats: mean=5.053915, std=0.040729, max=1
total degree stats: mean=10.107830, std=0.516658, max=1

*** PageRank
Max pageranks
88	Q30	0.025952
3049	Q336	0.014935
291	Q1860	0.013473
1767	Q151885	0.012089
1289	Q11862829	0.009268

*** HITS
HITS hubs
291	Q1860	0.708428
88	Q30	0.561884
98	Q5	0.222163
13	Q6581097	0.190558
554	Q723685	0.106949
HITS auth
3109	Q273055	0.028708
2437	Q202148	0.028366
20221	Q498019	0.028144
16214	Q40531	0.027984
10367	Q213512	0.027935


In [19]:
!kgtk query -i $TEMP/wd.kgtk.gz \
    --match '(n)-[r]->()' \
    --return 'count (distinct r.label)'

count(DISTINCT graph_37_c1."label")
617


What are the most common relations?

In [7]:
!kgtk query -i $TEMP/templates_with_wd.kgtk.gz \
    --match '(n)-[r]->()' \
    --return 'r.label, count(n) as c' \
    --order-by 'c desc' \
    -o $TEMP/rel_stats.tsv

Let's get nodes with highest indegree:

In [38]:
!kgtk query -i $TEMP/meme_stats.tsv \
    --match '(n1)-[:vertex_in_degree]->(n2)' \
    --return 'n1 as node1, printf("(%d),", n2) as node2' \
    --limit 10 \
    --order-by 'n2 desc'

node1	node2
Q277421	(99),
Q9633	(99),
Q42602	(99),
Q712378	(98),
youtube	(97),
Q4868296	(963),
Q42586	(96),
Q56	(95),
catchphrase	(95),
kymt:slang	(94),


## 5. Visualize instances

In [34]:
!kgtk query -i $TEMP/instances.kgtk.gz \
    --match '(:`imgflipi:1001ub`)-[r]->()' \
    --where 'r.label!="imgflipr:alt_text"' \
-o $TEMP/1001_ub.kgtk.gz

Create node file:

In [35]:
!kgtk query -i $TEMP/labelfile.kgtk.gz -i $TEMP/1001_ub.kgtk.gz \
            --match 'ub: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as id, l as label' / deduplicate \
            -o $TEMP/nodefile.kgtk.gz 

In [36]:
kgtk("""visualize-graph 
        -i $TEMP/1001_ub.kgtk.gz
        --node-file $TEMP/nodefile.kgtk.gz
        --show-text above
        --show-edge-label
        --tooltip-column label
        --direction arrow
        --edge-color-column label
        --edge-color-style d3.schemeDark2
        --show-text-limit 50
        -o viz/imgflip.graph.html""")

## 5. Visualize the full KG

In [9]:
kgtk("""visualize-graph 
        -i $TEMP/imkg.kgtk.gz
        -o full.graph.html""")

## 6. Graph statistics for IMKG

In [10]:
!kgtk graph-statistics \
     -i $TEMP/imkg.kgtk.gz \
     --log-file $TEMP/imkg_summary.txt \
     --output-statistics-only \
     -o $TEMP/imkg_stats.tsv


(kgtk:13430): Gtk-WARNING **: 21:15:40.937: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[13430]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x19f2f1960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1b36496b0). One of the two will be used. Which one is undefined.


In [11]:
!cat $TEMP/imkg_summary.txt

graph loaded! It has 646690 nodes and 2230499 edges

*** Top relations:
m4s:fromCaption	460177
imgflipr:alt_text	160461
imgflipr:image_url	160461
imgflipr:template	160461
imgflipr:templateId	160461
imgflipr:template_title	160461
imgflipr:title	160461
imgflipr:upvote_count	160461
imgflipr:view_count	160461
imgflipr:author	137923

*** Degrees:
in degree stats: mean=3.449101, std=0.301658, max=1
out degree stats: mean=3.449101, std=0.011192, max=1
total degree stats: mean=6.898202, std=0.301999, max=1

*** PageRank
Max pageranks
15	Q2927074	0.009397
101	Q30	0.006346
621526	Q336	0.005086
630289	Q9081	0.004848
1560	Bad Luck Brian	0.004195

*** HITS
HITS hubs
15	Q2927074	0.546047
1560	Bad Luck Brian	0.524021
503	Q18738659	0.290511
1558	imgflipmeme:Bad-Luck-Brian	0.290051
1559	61585	0.290043
HITS auth
545738	imgflipi:izyph	0.005110
46211	imgflipi:183a7b	0.005105
555539	imgflipi:l4j55	0.005103
303752	imgflipi:33jv3i	0.005095
600279	imgflipi:vumoh	0.005093


In [18]:
!kgtk query -i $TEMP/imkg.kgtk.gz \
    --match '()-[r]->()' \
    --return 'r.label, count(r.label) as c' \
    --order-by 'c desc' \
    --limit 20

label	c
m4s:fromCaption	460177
imgflipr:alt_text	160461
imgflipr:image_url	160461
imgflipr:template	160461
imgflipr:templateId	160461
imgflipr:template_title	160461
imgflipr:title	160461
imgflipr:upvote_count	160461
imgflipr:view_count	160461
imgflipr:author	137923
kym:sibling	111053
m4s:fromImage	53794
P31	18090
kym:tag	11050
P136	8558
P495	6942
P106	6030
P17	5572
m4s:fromAbout	5403
P407	4363


In [29]:
!kgtk query -i $TEMP/imkg.kgtk.gz -i $label \
    --match 'imkg: ()-[:`m4s:fromCaption`]->(n), \
            label: (n)-[]->(lbl)' \
    --return 'n, kgtk_lqstring_text_string(lbl), count(n) as c' \
    --order-by 'c desc' \
    --limit 10

node2	kgtk_lqstring_text_string(graph_17_c2."node2")	c
Q2927074	"Internet meme"	122634
Q18738659	"Kyle Craven"	36061
Q978	"meme"	23228
Q4946461	"spa town"	19938
Q622404	"boyfriend"	19874
Q25095883	"First World Problems"	15671
Q7714263	"Grumpy Cat"	12683
Q11661	"information technology"	10922
Q62141	"facepalm"	9677
Q16276	"Jean-Luc Picard"	8930


In [30]:
!kgtk query -i $TEMP/imkg.kgtk.gz -i $label \
    --match 'imkg: ()-[:`m4s:fromImage`]->(n), \
            label: (n)-[]->(lbl)' \
    --return 'n, kgtk_lqstring_text_string(lbl), count(n) as c' \
    --order-by 'c desc' \
    --limit 10

node2	kgtk_lqstring_text_string(graph_17_c2."node2")	c
Q2927074	"Internet meme"	1041
Q478798	"image"	1012
Q4868296	"font"	960
Q978	"meme"	840
Q2071334	"Know Your Meme"	824
Q735	"art"	714
Q178659	"illustration"	609
Q8	"happiness"	598
Q1027879	"graphics"	596
Q371174	"gesture"	546


In [31]:
!kgtk query -i $TEMP/imkg.kgtk.gz -i $label \
    --match 'imkg: ()-[:`m4s:fromAbout`]->(n), \
            label: (n)-[]->(lbl)' \
    --return 'n, kgtk_lqstring_text_string(lbl), count(n) as c' \
    --order-by 'c desc' \
    --limit 10

node2	kgtk_lqstring_text_string(graph_17_c2."node2")	c
Q6002242	"image macro"	216
Q866	"YouTube"	100
Q2927074	"Internet meme"	78
Q238330	"4chan"	74
Q2708515	"catchphrase"	70
Q83279	"SpongeBob SquarePants"	66
Q75	"Internet"	53
Q43747	"Internet slang"	52
Q918	"Twitter"	52
Q8102	"slang"	46


In [32]:
!kgtk query -i $TEMP/imkg.kgtk.gz -i $label \
    --match 'imkg: ()-[:`m4s:fromTags`]->(n), \
            label: (n)-[]->(lbl)' \
    --return 'n, kgtk_lqstring_text_string(lbl), count(n) as c' \
    --order-by 'c desc' \
    --limit 10

node2	kgtk_lqstring_text_string(graph_17_c2."node2")	c
Q6002242	"image macro"	167
Q238330	"4chan"	128
Q2708515	"catchphrase"	95
Q8102	"slang"	79
Q144	"dog"	65
Q1136	"Reddit"	44
Q3002431	"creepypasta"	42
Q2927074	"Internet meme"	41
Q384060	"Tumblr"	39
Q1030329	"viral video"	36


In [26]:
!kgtk graph-statistics \
     -i $TEMP/combined.kgtk.gz \
     --log-file $TEMP/combined_summary.txt \
     --output-statistics-only \
     -o $TEMP/combined_stats.tsv


(kgtk:13891): Gtk-WARNING **: 21:39:56.356: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[13891]: Class GNotificationCenterDelegate is implemented in both /Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/libgio-2.0.0.dylib (0x19cb35960) and /usr/local/Cellar/glib/2.72.2/lib/libgio-2.0.0.dylib (0x1a80c16b0). One of the two will be used. Which one is undefined.


In [27]:
!cat $TEMP/combined_summary.txt

graph loaded! It has 646109 nodes and 2093922 edges

*** Top relations:
m4s:fromCaption	460177
imgflipr:alt_text	160461
imgflipr:image_url	160461
imgflipr:template	160461
imgflipr:templateId	160461
imgflipr:template_title	160461
imgflipr:title	160461
imgflipr:upvote_count	160461
imgflipr:view_count	160461
imgflipr:author	137923

*** Degrees:
in degree stats: mean=3.240818, std=0.300632, max=1
out degree stats: mean=3.240818, std=0.011095, max=1
total degree stats: mean=6.481637, std=0.300917, max=1

*** PageRank
Max pageranks
499	Q2927074	0.011136
641	Bad Luck Brian	0.005408
141724	Distracted Boyfriend	0.004409
518	Disaster Girl	0.003240
156	Q18738659	0.003121

*** HITS
HITS hubs
499	Q2927074	0.544366
641	Bad Luck Brian	0.524943
156	Q18738659	0.291014
639	imgflipmeme:Bad-Luck-Brian	0.290554
640	61585	0.290546
HITS auth
544994	imgflipi:izyph	0.005117
45347	imgflipi:183a7b	0.005111
554795	imgflipi:l4j55	0.005109
302974	imgflipi:33jv3i	0.005101
599536	imgflipi:vumoh	0.005100
